In [ ]:
import cv2
import numpy as np
import dlib
import os

size = 4
haar_face_cascade = 'haarcascade_frontalface_default.xml'
datasets = r'C:\Users\aanki\Anti spooling\datasets'  # Adjust this path as needed

# Load Dlib's pre-trained shape predictor model
shape_predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye_points):
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Part 1: Create LBPHFaceRecognizer
print('Recognizing Face. Please be in sufficient light...')

# Create a list of images and a list of corresponding names
(images, labels, names, id) = ([], [], {}, 0)
for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = os.path.join(subjectpath, filename)
            img = cv2.imread(path, 0)
            if img is not None:  # Ensure the image was loaded
                img = cv2.resize(img, (130, 100))  # Resize images
                images.append(img)
                labels.append(id)
        id += 1

(width, height) = (130, 100)

# Create a Numpy array from the two lists above
images = np.array(images)
labels = np.array(labels)

# OpenCV trains a model from the images
model = cv2.face.LBPHFaceRecognizer_create()
model.train(images, labels)

# Part 2: Use LBPHFaceRecognizer on camera stream
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + haar_face_cascade)

webcam = cv2.VideoCapture(0)

# Parameters for motion detection and blink detection
ret, prev_frame = webcam.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
motion_threshold = 10000  # Adjust this threshold based on your testing
blink_counter = 0
total_blinks = 0
blink_threshold = 5  # Adjust based on typical blink behavior
confidence_threshold = 70  # Threshold for considering a face as recognized
required_blinks = 3  # Number of blinks required to declare as real

blink_counts = {}  # Dictionary to keep track of blinks for each detected face

while True:
    ret, frame = webcam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Check for motion
    diff_frame = cv2.absdiff(prev_gray, gray)
    _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
    motion = np.sum(thresh_frame)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))
        prediction = model.predict(face_resize)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        if prediction[1] < confidence_threshold:
            name = names[prediction[0]]
            status = "Registered"
            color = (0, 255, 0)  # Green for registered
        else:
            name = "Not Registered"
            status = "Not Registered"
            color = (0, 0, 255)  # Red for not registered

        cv2.putText(frame, f'{name}', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color)
        cv2.putText(frame, status, (x - 10, y + h + 10), cv2.FONT_HERSHEY_PLAIN, 1, color)

        # Convert the face region back to a format dlib can work with
        dlib_rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        shape = predictor(gray, dlib_rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])

        left_eye = shape[36:42]
        right_eye = shape[42:48]

        ear_left = eye_aspect_ratio(left_eye)
        ear_right = eye_aspect_ratio(right_eye)

        if (ear_left + ear_right) / 2.0 < 0.25:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                total_blinks += 1
                blink_counter = 0

                if prediction[0] in blink_counts:
                    blink_counts[prediction[0]] += 1
                else:
                    blink_counts[prediction[0]] = 1

        # Liveness detection
        if name != "Not Registered" and blink_counts.get(prediction[0], 0) >= required_blinks:
            liveness_status = "Real"
            liveness_color = (0, 255, 0)  # Green for real
        elif motion < motion_threshold:
            liveness_status = "Fake"
            liveness_color = (0, 0, 255)  # Red for fake
        else:
            liveness_status = "Undetermined"
            liveness_color = (0, 255, 255)  # Yellow for undetermined

        cv2.putText(frame, liveness_status, (x - 10, y + h + 20), cv2.FONT_HERSHEY_PLAIN, 1, liveness_color)

    cv2.imshow('OpenCV', frame)
    prev_gray = gray

    key = cv2.waitKey(10)
    if key == 27:
        break

webcam.release()
cv2.destroyAllWindows()


In [15]:
import cv2
import numpy as np
import dlib
import os

size = 4
haar_face_cascade = 'haarcascade_frontalface_default.xml'
datasets = r'C:\Users\aanki\Anti spooling\datasets'  # Adjust this path as needed

# Load Dlib's pre-trained shape predictor model
shape_predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye_points):
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Part 1: Create LBPHFaceRecognizer
print('Recognizing Face. Please be in sufficient light...')

# Create a list of images and a list of corresponding names
(images, labels, names, id) = ([], [], {}, 0)
for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = os.path.join(subjectpath, filename)
            img = cv2.imread(path, 0)
            if img is not None:  # Ensure the image was loaded
                img = cv2.resize(img, (130, 100))  # Resize images
                images.append(img)
                labels.append(id)
        id += 1

(width, height) = (130, 100)

# Create a Numpy array from the two lists above
images = np.array(images)
labels = np.array(labels)

# OpenCV trains a model from the images
model = cv2.face.LBPHFaceRecognizer_create()
model.train(images, labels)

# Part 2: Use LBPHFaceRecognizer on camera stream
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + haar_face_cascade)

webcam = cv2.VideoCapture(0)

# Parameters for motion detection and blink detection
ret, prev_frame = webcam.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
motion_threshold = 10000  # Adjust this threshold based on your testing
blink_counter = 0
total_blinks = 0
blink_threshold = 5  # Adjust based on typical blink behavior
confidence_threshold = 70  # Threshold for considering a face as recognized
required_blinks = 3  # Number of blinks required to declare as real

blink_counts = {}  # Dictionary to keep track of blinks for each detected face
real_face_count = 0
fake_or_undetermined_count = 0

while True:
    ret, frame = webcam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Check for motion
    diff_frame = cv2.absdiff(prev_gray, gray)
    _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
    motion = np.sum(thresh_frame)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))
        prediction = model.predict(face_resize)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        if prediction[1] < confidence_threshold:
            name = names[prediction[0]]
            status = "Registered"
            color = (0, 255, 0)  # Green for registered
        else:
            name = "Not Registered"
            status = "Not Registered"
            color = (0, 0, 255)  # Red for not registered

        cv2.putText(frame, f'{name}', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color)
        cv2.putText(frame, status, (x - 10, y + h + 10), cv2.FONT_HERSHEY_PLAIN, 1, color)

        # Convert the face region back to a format dlib can work with
        dlib_rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        shape = predictor(gray, dlib_rect)
        shape = np.array([[p.x, p.y] for p in shape.parts()])

        left_eye = shape[36:42]
        right_eye = shape[42:48]

        ear_left = eye_aspect_ratio(left_eye)
        ear_right = eye_aspect_ratio(right_eye)

        if (ear_left + ear_right) / 2.0 < 0.25:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                total_blinks += 1
                blink_counter = 0

                if prediction[0] in blink_counts:
                    blink_counts[prediction[0]] += 1
                else:
                    blink_counts[prediction[0]] = 1

        # Liveness detection
        if name != "Not Registered" and blink_counts.get(prediction[0], 0) >= required_blinks:
            liveness_status = "Real"
            liveness_color = (0, 255, 0)  # Green for real
            real_face_count += 1
        elif motion < motion_threshold:
            liveness_status = "Fake"
            liveness_color = (0, 0, 255)  # Red for fake
            fake_or_undetermined_count += 1
        else:
            liveness_status = "Undetermined"
            liveness_color = (0, 255, 255)  # Yellow for undetermined
            fake_or_undetermined_count += 1

        cv2.putText(frame, liveness_status, (x - 10, y + h + 20), cv2.FONT_HERSHEY_PLAIN, 1, liveness_color)

    cv2.imshow('OpenCV', frame)
    prev_gray = gray

    key = cv2.waitKey(10)
    if key == 27:
        break

    if real_face_count >= 100:
        print("Live Detected")
        break
    elif fake_or_undetermined_count >= 100:
        print("Can't Identify")
        break

webcam.release()
cv2.destroyAllWindows()


Recognizing Face. Please be in sufficient light...
Can't Identify
